In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from torch.optim.lr_scheduler import ReduceLROnPlateau

import cv2
from skimage import io, exposure
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import bbox_visualizer as bbv

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from glob import glob
from skimage import exposure

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torchvision.ops import box_iou

import json
import numpy as np

import warnings

warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_csv(os.path.join("train1.csv"))
dataset.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332.0,2580.0
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954.0,3159.0
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,0.332212,0.588613,0.794712,0.783818,2080.0,2336.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,0.548611,0.257986,0.699219,0.353819,2304.0,2880.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540.0,3072.0


In [3]:
dataset_new = dataset[dataset.class_name!='No finding'].reset_index(drop=True)

In [4]:
num_classes = 15

In [5]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [6]:
def set_device():
    device = "cpu"
    return device

device=set_device()

In [7]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]

In [8]:
def get_valid_transform():
    return A.Compose([ToTensorV2(),], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [9]:
class LungsAnnotationDataset(Dataset):
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()
        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]

        image = io.imread(f'{self.image_dir}/{image_id}.png')

        # Normalize the image
        image = image / 255.0  # pixel values are in the range [0, 255]
        image = exposure.equalize_hist(image)
        image = image.astype('float32')

        # If the image has 3 channels already (like RGB), no need to stack, else ensure 3 channels
        if image.ndim == 2:  # If the image is grayscale, convert to 3 channels
            image = np.stack([image, image, image], axis=-1)

        # Ensure the image is in the correct (C, H, W) format
        if image.shape[2] == 3:  # Check if image is in (H, W, C)
            image = image.transpose(2, 0, 1)  # Convert from (H, W, C) to (C, H, W)

        # Get bounding boxes and other details
        boxes = records[['x_min', 'y_min', 'x_max', 'y_max']].values

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        labels = records.class_id.values + 1
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)

        target = {
            'boxes': torch.tensor(boxes, dtype=torch.float32),
            'labels': torch.tensor(labels, dtype=torch.int64),
            'area': area,
            'iscrowd': iscrowd
        }

        # Apply transformations if available (pass normalized boxes)
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = torch.tensor(sample['bboxes'], dtype=torch.float32)

        # Denormalize boxes AFTER transformations (if you need pixel coordinates)
        target['boxes'][:, [0, 2]] = target['boxes'][:, [0, 2]] * 512
        target['boxes'][:, [1, 3]] = target['boxes'][:, [1, 3]] * 512

        return image, target


    def __len__(self):
        return len(self.image_ids)

In [10]:
def collate_fn(batch):
    images, targets = zip(*batch)
    images = [image.permute(1, 2, 0) if image.shape[0] != 3 else image for image in images]
    return torch.stack(images), targets


In [11]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [12]:
class_brands = {
    0: 'Aortic enlargement',
    1: 'Atelectasis',
    2: 'Calcification',
    3: 'Cardiomegaly',
    4: 'Consolidation',
    5: 'ILD',
    6: 'Infiltration',
    7: 'Lung Opacity',
    8: 'Nodule/Mass',
    9: 'Other lesion',
    10: 'Pleural effusion',
    11: 'Pleural thickening',
    12: 'Pneumothorax',
    13: 'Pulmonary fibrosis'
}

In [13]:
val_dataset = LungsAnnotationDataset(dataframe=dataset_new, image_dir='resized_images', transforms=get_valid_transform())
val_data_loader = DataLoader(val_dataset, batch_size=20, shuffle=False, collate_fn=collate_fn)

In [14]:
print(len(val_data_loader))

220


In [15]:
model.load_state_dict(torch.load('x_ray_models/model_fasterRCNN_finetuned.pth', map_location=device), strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['roi_heads.box_predictor.fc.weight', 'roi_heads.box_predictor.fc.bias'])

In [16]:
def save_predictions_to_json(model, val_data_loader, device, class_brands, filename='phase1_results.json'):
    results = {}
    
    model.to(device)  # Ensure the model is on the same device as the inputs
    model.eval()
    with torch.no_grad():
        for idx, (images, _) in enumerate(val_data_loader):
            images = [image.to(device) for image in images]  # Move images to the same device
            
            outputs = model(images)  # Model and inputs are on the same device
            
            for i, output in enumerate(outputs):
                image_id = f'image_{idx * len(outputs) + i}'  # Generate a unique ID for each image
                
                pred_boxes = output['boxes']
                labels_pred = output['labels']
                scores = output['scores'].data.cpu().numpy()

                # Filter predicted boxes based on confidence score
                valid_indices = scores >= 0.2
                boxes_pred = pred_boxes[valid_indices]
                labels_pred = labels_pred[valid_indices]

                # Convert results to list of dictionaries
                pred_results = []
                for box, label in zip(boxes_pred, labels_pred):
                    box_np = box.detach().cpu().numpy().astype(int).tolist()  # Convert tensor to list
                    class_name = class_brands.get(label.item(), 'Unknown')
                    pred_results.append({
                        'box': box_np,
                        'class_label': class_name
                    })

                results[image_id] = pred_results
    
    # Write results to a JSON file
    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)

    print(f"Results saved to {filename}")

In [17]:
def set_device():
    device =  "cpu"
    return device

device=set_device()

In [18]:
save_predictions_to_json(model, val_data_loader, device, class_brands)

Results saved to phase1_results.json


In [19]:
# iou_hist = Averager()
# model.eval()
# with torch.no_grad():
#     for images, targets in val_data_loader:
#         images = [image.to(device) for image in images]
#         targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

#         outputs = model(images)

#         for output, target in zip(outputs, targets):
#             pred_boxes = output['boxes']
#             gt_boxes = target['boxes']

#             if pred_boxes.numel() == 0 or gt_boxes.numel() == 0:
#                 print(f"Skipping due to empty predicted or ground truth boxes")
#                 continue

#             # Calculate IoU
#             iou = box_iou(pred_boxes, gt_boxes)

#             # Handle NaN values
#             if torch.isnan(iou).any():
#                 print(f"NaN detected in IoU calculation. Skipping this batch.")
#                 continue

#             # Append average IoU for each image
#             avg_iou = iou.mean().item()
#             iou_hist.send(avg_iou)

#     avg_iou = iou_hist.value
#     print(f"Validation IoU: {avg_iou:.4f}")




